In [122]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split,RandomizedSearchCV,cross_val_score,GridSearchCV
from sklearn.utils import resample
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

In [123]:
df=pd.read_csv("../data/df_4classes.csv", index_col=[0])
medal_df=df[["id","medal"]]
df.drop(["event","medal"],axis=1,inplace=True)

In [137]:
medal.reset_index(drop=True)

,id,medal
0,18289,No medal
1,57077,No medal
2,8528,No medal
3,82322,No medal
4,10102,No medal
...,...,...
41471,135553,Silver
41472,135553,Bronze
41473,135554,Bronze
41474,135563,Bronze


In [125]:
numeric_df=df.select_dtypes(exclude="object")
cat_df=df.select_dtypes(include="object")

In [126]:
cat_df

,sex,noc,season,city,sport
35564,F,ANG,Summer,Beijing,Handball
112673,F,POL,Summer,Moskva,Canoeing
15820,M,ROU,Summer,London,Gymnastics
164015,M,FRA,Summer,Athina,Rowing
19092,F,GER,Summer,Sydney,Rhythmic Gymnastics
...,...,...,...,...,...
271078,F,URS,Summer,Melbourne,Athletics
271080,F,URS,Summer,Tokyo,Athletics
271082,M,POL,Summer,Moskva,Fencing
271102,F,RUS,Summer,Sydney,Athletics


In [127]:
knn_imp = KNNImputer(n_neighbors=5)
numeric_df = pd.DataFrame(knn_imp.fit_transform(numeric_df),
                           columns=numeric_df.columns)

In [128]:
cat_cols=cat_df.columns
cat_dummy_encoder = OneHotEncoder(handle_unknown='ignore')
df_cat_ohe = cat_dummy_encoder.fit_transform(cat_df).toarray()
cat_ohe_cols = cat_dummy_encoder.get_feature_names(cat_cols)
df_cat_ohe = pd.DataFrame(df_cat_ohe,
                               columns=cat_ohe_cols)

In [129]:
df_cat_ohe

,sex_F,sex_M,noc_AFG,noc_AHO,noc_ALB,noc_ALG,noc_AND,noc_ANG,noc_ANT,noc_ARG,...,sport_Synchronized Swimming,sport_Table Tennis,sport_Taekwondo,sport_Tennis,sport_Trampolining,sport_Triathlon,sport_Volleyball,sport_Water Polo,sport_Weightlifting,sport_Wrestling
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41471,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
41472,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
41473,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
41474,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [130]:
df=pd.concat([numeric_df,df_cat_ohe],axis=1)

In [131]:
df.reset_index(drop=True)

,id,age,height,weight,year,sex_F,sex_M,noc_AFG,noc_AHO,noc_ALB,...,sport_Synchronized Swimming,sport_Table Tennis,sport_Taekwondo,sport_Tennis,sport_Trampolining,sport_Triathlon,sport_Volleyball,sport_Water Polo,sport_Weightlifting,sport_Wrestling
0,18289.0,18.0,175.0,69.0,2008.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,57077.0,27.0,164.0,69.0,1980.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,8528.0,24.0,171.0,62.0,2012.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,82322.0,22.0,195.0,87.0,2004.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,10102.0,18.0,160.0,43.0,2000.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41471,135553.0,25.0,168.0,80.0,1956.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
41472,135553.0,33.0,168.0,80.0,1964.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
41473,135554.0,28.0,182.0,82.0,1980.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
41474,135563.0,19.0,171.0,64.0,2000.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [134]:
df["medal"]=medal_df.medal

In [135]:
df

,id,age,height,weight,year,sex_F,sex_M,noc_AFG,noc_AHO,noc_ALB,...,sport_Table Tennis,sport_Taekwondo,sport_Tennis,sport_Trampolining,sport_Triathlon,sport_Volleyball,sport_Water Polo,sport_Weightlifting,sport_Wrestling,medal
0,18289.0,18.0,175.0,69.0,2008.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,No medal
1,57077.0,27.0,164.0,69.0,1980.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,No medal
2,8528.0,24.0,171.0,62.0,2012.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,No medal
3,82322.0,22.0,195.0,87.0,2004.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,No medal
4,10102.0,18.0,160.0,43.0,2000.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,No medal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41471,135553.0,25.0,168.0,80.0,1956.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Silver
41472,135553.0,33.0,168.0,80.0,1964.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Bronze
41473,135554.0,28.0,182.0,82.0,1980.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Bronze
41474,135563.0,19.0,171.0,64.0,2000.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Bronze


In [ ]:
df.to_csv("../data.csv")